
# Libraries and Tools used

In [3]:
"""Our graphical elements are going to be done with matplotlib, inline graphs"""
%matplotlib inline
import matplotlib.pyplot as plt

"""CSV: utility to work with the most common format for datastorage : Comma Separated Values"""
import csv

"""JSON: utility to work with JavaScript Object Notation, the original information format storage we """
import json
from collections import namedtuple

"""Pandas: python's most used library to work with datasets"""
import pandas as pd 

""" Type detection for our dataframe """
from pandas.api.types import (
    is_categorical_dtype,
    is_datetime64_dtype,
    is_object_dtype,
    is_numeric_dtype,
    is_string_dtype
)

"""Numpy: python's most used library to work with large amounts of numbers"""
import numpy as np
"""OS: utility to work with paths and file openings independently of operating system"""
import os
"""Glob: used to look for file extensions inside given folders"""
import glob 

"""Candidates for ML predictive model implementations"""
import tensorflow as tf
import sklearn
import catboost

# The Storage module

### Methods 

#### __init__
#### print_info
#### from_json
#### to_json
#### update_stats
#### load_from_dataframe


In [4]:
class StoreDomain():
    
    def __init__(self, attribute_list, path=os.getcwd(), domain_name=None):
        """ Handy initialization """
        self.load(attribute_list, path, domain_name)
        
    def load(self, attribute_list, path, domain_name):
        """ Given a dataset searches for a domain in the specified folder.
            If found, loads it into the class itself. If not found loads an empty template.
        """
        for fnm in glob.glob(os.path.join(path,"*.json")):
            with open(fnm) as f:
                data = json.load(f)
                print("     ", data["attribute_list"], attribute_list)
            if data["attribute_list"] == attribute_list:
                self.fnm = fnm
                self.domain_name = domain_name or str(attribute_list)
                self.modify(data)
                break;
                        
        if not hasattr(self,"fnm"):
            print("No matching knowledge found for your domain, setting an empty one")
            self.attribute_list = attribute_list
            self.domain_name = domain_name or str(attribute_list)
            self.knowledge = {
                "dataset_stats" : {},
                "column_stats": {}
            }
            self.fnm = glob.glob(os.path.join( path, str(self.domain_name)+".json") )
                    
    def save(self):
        """ Saves itself as an object in .json format
        """
        print("" "", self.fnm)
        with open(self.fnm,'w') as f:
            json.dump(self.__dict__, f)
            
    def modify(self,data):
        """ Modifies the class from a physical representation (dict) of it
        """   
        self.domain_name = data["domain_name"] if "domain_name" in data else self.domain_name
        self.attribute_list = data["attribute_list"] if "attribute_list" in data else self.attribute_list
        self.domain_knowledge = data["domain_knowledge"] if "domain_knowledge" in data else self.domain_knowledge
        self.fnm = data["fnm"] if "fnm" in data else self.fnm

    def print_info(self):
        """ Util to print attributes
        """
        print(self.__dict__)


# The JSON
&nbsp;

 This object is used to represent the raw information stored.

 It contains statistics and properties from both columns and datasets, different for different types, as well as how they were measured (by saving the function to be called on a pandas dataset in the case of a dataset stat, or on the values themselves in the case of a function stat).

 Saving how they're measured is important to measure new datasets and to be able to compare metrics effectively.

 Both the column specific stats and the dataset stats are subjective to change, and the knowledgedomain object can be modified and adapted to fit new parameters easily.

 This structure is generated for a single dataset and then combined with the domain one to take account of the new one.



In [5]:
KNOWLEDGE_STRUCTURE = {
    "dataset_stats": {
        "individuals": 0,
        "max_rows": {
            "value": 1400,
            "metric" : "count_rows_null",
        }, # alert if new rows are <<<, as different results are skewed
        "avg_rows": {
            "value": 1400,
            "metric" : "count_rows_null"
        },
        "full_rows": {
            "value": 0.97
        }
    },
    "column_stats": {
        "a": {
            "type": "numeric",
            "stats": {
                "median": 12.6,
                "std_dev": 1.3,
                "max": 25.5,
                "min": 2.3,
                "NaNs": 0.02 # as % of dataset
            }     
        },
        "b": {
            "type": "categorical",
            "stats": {
                "most_frequent": "cat",
                "values": {
                    "cat": 0.2,
                    "dog": 0.6
                },
                "nan": 0.1
            }
            
        }

    }
}


# The analysis module
## Metrics

In [6]:
"""
    The purpose of these functions is to provide a way to measure the properties of a given dataset or knowledge domain.
    We can categorize them as follows:
    First the "measurement" metrics, used to get the information of a single dataset or domain.
    
    - Dataset Metrics : they concern the dataset as a whole, like number of rows with missing values.
        dm :: (ds) --> num
        
    - Single Column Metrics : they concern a certain column, and are based on the type of the column.
      For numerical columns we will have things like median, averages, deviations, distributions...
      For categorical columns we'll work with frequencies and things of the sort.
        scm :: (col) --> num
        
    - Multiple Column Metrics : we will be looking for correlations and things of that sort.
       scm :: (col,col) --> num 
       Time based metrics will be defined from this construct.
    
    We will also have "comparison" metrics, used to compare datasets against their domains.
    These metrics will compare the output of two measurement metrics, both will have to
    spawn from the same function.
        comp_m :: (metric) --> num
    
    Note that these metrics are not to provide "meaning" or any human-readable input, nor to be
    inherently comparable between each other outside of a framework of understanding of the domain
    (metric importance).
    
    A mean to convert these machine cold metrics into human understanding will be provided in further 
    modules. For now, we're not taking humans into account.
"""

""" Dataset Metrics """

def count_rows_null(df):
    return df.isnull().shape(0)

""" Columns """

""" For consistency, every column is accepted as a pandas Series """
""" Numeric, accepted as pandas Series """

def median(col):
    return np.median(col.values)

def average(col):
    return np.average(col.values)



""" All types """

def count_not_null(col):
    return np.count_nonzero(~np.isnan(data))

def count_null(col):
    return col.size - count_not_null(col)



""" Categorical """

def count_freqs(col):
    return 1.2
def count_nans(col):
    return 0.9



## Utility functions

In [7]:
def num_metric(a):
    return median_raw(a)
def median_raw(a):
    return np.median(a)
def most_frequent(a):
    return np.bincount(a).argmax()
IS_DTYPE = {
    "number": is_numeric_dtype,
    "category": is_categorical_dtype,
    "datetime": is_datetime64_dtype,
}

DEFAULT_METRICS = { #(type,comparison_metric)
    "numerical" : median_raw,
    "categorical": most_frequent,
    "dataset" : {
        "count_rows":{
            "metric": 2
        },
        "rows_with_nulls":{
            "metric": 3
        }
    }
}

COMPARISON_METRICS = { #(metric(name of a function) , comparison_metric)
    
}

In [8]:
class DatasetAnalyzer():
    
    def __init__(self, df, path):
        self.dataset = df            
        self.std = StoreDomain(domain_name=df.name, attribute_list=list(df.columns), path=path)
        self.domain_knowledge = self.std.domain_knowledge
        self.dataset_knowledge = self.get_stats(df)
        
    def get_stats(self, df):
        """ Populates the stats dictionary """
        return {
            "column_stats": self.get_column_stats(df),
            "dataset_stats": self.get_dataset_stats(df)
        }
    
    def get_column_stats(self,df):
        """ Column stats """

        stats = {}
        col_types = self.get_column_types(df)
        
        for c in self.get_column_types(df):
            if c in self.domain_knowledge["column_stats"]:
                func = globals()[self.domain_knowledge["column_stats"][c]["metric"]]
            else:
                func = DEFAULT_METRICS[col_types[c]]
                
            stats[c] = {
                    "metric" : func.__name__,
                    "value" : func(df[c].values)
            }
            
        return stats
    
    def get_dataset_stats(self,df):
        """ Dataset stats """
        stats = {}
        dataset_metrics = self.domain_knowledge["dataset_stats"]
        for m in dataset_metrics:
            stats[m] = {
                "metric": globals()[dataset_metrics[m]["metric"]],
                "value": globals()[dataset_metrics[m]["metric"]](df)
            }
            
        return stats
    
    def get_column_types(self, df):
        """ Get column types """
        col_types = {}
        for c in df.columns:
            col_types[c] = "numerical" if IS_DTYPE['number'](df[c].dtype) else "categorical"
            
        return col_types
    
    def fill_d1_with_d2(self, d1, d2):
        """Add extra stats from d2 (default) to d1 (our dict)"""
        for key in d2:
            if (key in d1 and isinstance(d1[key], dict) and isinstance(d2[key], dict)):
                self.deep_merge_dicts(d1[key], d2[key])
            else:
                d1[key] = d2[key]

    def get_analysis(self): #generates report, saves it in self.report AND returns it.
        """ This method will return a JSON containing the comparison between the dataset knowledge
            and the domain knowledge
        """    

        col_stats_comparison = {
            "column_stats": self.compare_stats(self.domain_knowledge["column_stats"], self.dataset_knowledge["column_stats"])
        }
        
        dataset_stats_comparison = {
            "dataset_stats": self.compare_stats(self.domain_knowledge["dataset_stats"], self.dataset_knowledge["dataset_stats"])
        }
        comparison = dict(col_stats_comparison, **dataset_stats_comparison)
        for key in comparison["column_stats"]:
            comparison["column_stats"][key] = {
                "value": comparison["column_stats"][key],
                "score": "relevant" if comparison["column_stats"][key] > 0.5 else "not_relevant"
            }

        comparison = dict(col_stats_comparison, **dataset_stats_comparison)
        """TODO: FIND BETTER NAMES FFS"""

        return dict(col_stats_comparison, **dataset_stats_comparison)
    
    def compare_stats(self,stats1, stats2):
        """ Compares two dictionaries with the same structure """
        comparison = {}

        if stats1:
            for c in stats1:
                if(stats1[c]["metric"] == stats1[c]["metric"]):
                    #TODO: use a real comparison metric
                    comparison[c] = stats2[c]["value"] - stats2[c]["value"]
            return comparison
        else:
            return stats2
    
    def update_stats(self): 
        new_stats = {}
        #TODO : customize this so weights can be other things aswell
        n = self.domain_knowledge["n"]
        for c in self.dataset_knowledge:
            self.domain_knowledge[c]["value"] = (n * self.domain_knowledge[c]["value"] + self.dataset_knowledge[c]["value"]) / (n + 1)
        
        #TODO : customize this so you're able to take into account the number of rows
        self.domain_knowledge["n"] = n + 1
        
        #Save the information
        self.std.knowledge = self.domain_knowledge


# CBR Database

In [9]:
class StoreCBR(StoreDomain):
    """ StoreDomain but for human knowledge. Extends StoreDomain to add profile functionality
        and CBR-based behaviour.
        Each storecbr is tied to a profile and it can only modify it
    """
    def __init__(self, attribute_list, profile, path, domain_name=None):
        """ Handy initialization """
        self.load(attribute_list, profile, path, domain_name)
        
    def load(self, attribute_list, profile, path, domain_name):
        """ Given a dataset searches for a domain in the specified folder(s) and loads
        """
        for fnm in glob.glob(os.path.join(path,"*.json")):
            with open(fnm) as f:
                data = json.load(f)
            if data["attribute_list"] == attribute_list:
                self.profile = profile
                self.domain_name = data["domain_name"] if "domain_name" in data else str(attribute_list)
                self.attribute_list = data["attribute_list"] if "attribute_list" in data else self.attribute_list
                self.profiles = data["profiles"] 
                self.fnm = fnm
                break;
                        
        if not hasattr(self,"fnm"):
            self.set_defaults(domain_name ,attribute_list, path, profile)
            
    def set_defaults(self, domain_name ,attribute_list, path, profile):
        """
        """
        self.domain_name = domain_name or attribute_list
        self.attribute_list = attribute_list
        self.profile = profile

        self.fnm = glob.glob(os.path.join( path, self.domain_name+".json") )
        self.knowledge = {
            "profiles": {
                "default" : {},
                profile : {
                    "profile_knowledge" : {
                        "column_stats": {},
                        "dataset_stats": {}
                    },
                    "graphical_knowledge": {}
                }
            }
        }
        
        #self.knowledge["profiles"][profile] = {
        #    "profile_knowledge" : {}
        #}
        
    def modify(self, new_info):
        self.profiles[self.profile] = new_info
        
    def run_tournament(self): pass
    """ELO """    
    def interaction_learning(self): pass
    """ despues"""

# Reporter

In [10]:
""" 
    Another kind of information is stored about the domains. This is the 
    information concerning the **human** side of things, that is, **how** 
    to interpret these stats and turn them into something that humans with different levels of 
    familiarity can understand.
    To do this, we provide use another storage class that will contain human-relevant data that
    will modify the objective comparison delivered by the analysis module.
"""

class Reporter():
    
    def __init__(self, df, profile, domain_path, human_knowledge_path):
        self.profile = profile
        self.dsa = DatasetAnalyzer(df, path=domain_path)
        self.scbr = StoreCBR(list(df.columns), profile, path=human_knowledge_path, domain_name=df.name)
        self.generate()
        
    def generate(self):

        self.analysis = self.dsa.get_analysis()
        self.dataset_data = self.dsa.dataset_knowledge
        self.human_knowledge = self.scbr.profiles[self.scbr.profile]
        self.report = {"dataset":{},"columns":{},"dataset_profile":{},"columns_profile":{}, "graphical_information":{}}
        print(self.analysis)
        """Comparison between domain data and profile data : how much does this user care about..."""
        for k in self.analysis["dataset_stats"].keys():
            self.report["dataset"][k] = {
                "value": self.analysis["dataset_stats"][k]["value"] * self.human_knowledge["profile_knowledge"]["dataset_stats"][k]["value"],
                "score": "TODO"
            }
            
        for k in self.analysis["column_stats"].keys():
            self.report["columns"][k] = {
                "value": self.analysis["column_stats"][k]["value"] * self.human_knowledge["profile_knowledge"]["column_stats"][k]["value"],
                "score": "TODO"
            }
            
        """Comparison between profile data and profile data : analysis but user vs itself historical..."""
        for k in self.human_knowledge["profile_knowledge"]["dataset_stats"].keys():
            self.report["dataset_profile"][k] = {
                "value": self.human_knowledge["profile_knowledge"]["dataset_stats"][k]["value"] * self.dataset_data["dataset_stats"][k]["value"],
                "score": "TODO"

            }
        
        for k in self.human_knowledge["profile_knowledge"]["column_stats"].keys():
            self.report["columns_profile"][k] = {
                "value": self.human_knowledge["profile_knowledge"]["column_stats"][k]["value"] * self.dataset_data["column_stats"][k]["value"],
                "score": "TODO"

            }
        """ Graphical Information """
        for k in self.human_knowledge["graphical_knowledge"].keys():
            self.report["graphical_information"][k] = {
                "color" : self.human_knowledge["graphical_knowledge"][k]["color"],
                "graph" :  self.human_knowledge["graphical_knowledge"][k]["graph"]
            }

    def modify(self,new_info):
        self.human_info.modify(new_info)
    def save_human_info(self,new_info):
        self.human_info.save(new_info)

# Cardio Test

In [17]:
%%time
df = pd.read_csv("data/cardio_train.csv",sep=";")
df.name = "medical_data"

#Set cols as categorical
cat_cols = ["cholesterol", "gluc", "smoke",  "alco", "active", "cardio", "gender"]

for c in cat_cols:
    df[c] = df[c].astype("category")
df.head()
repo = Reporter(df, profile="patient", domain_path="./domain_storage/", human_knowledge_path="./human_storage/")


      ['id', 'age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc', 'smoke', 'alco', 'active', 'cardio'] ['id', 'age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc', 'smoke', 'alco', 'active', 'cardio']
{'column_stats': {'id': {'value': 0.0, 'score': 'not_relevant'}, 'age': {'value': 0.0, 'score': 'not_relevant'}, 'gender': {'value': 0.0, 'score': 'not_relevant'}, 'height': {'value': 0.0, 'score': 'not_relevant'}, 'weight': {'value': 0.0, 'score': 'not_relevant'}, 'ap_hi': {'value': 0.0, 'score': 'not_relevant'}, 'ap_lo': {'value': 0.0, 'score': 'not_relevant'}, 'cholesterol': {'value': 0.0, 'score': 'not_relevant'}, 'gluc': {'value': 0.0, 'score': 'not_relevant'}, 'smoke': {'value': 0.0, 'score': 'not_relevant'}, 'alco': {'value': 0.0, 'score': 'not_relevant'}, 'active': {'value': 0.0, 'score': 'not_relevant'}, 'cardio': {'value': 0.0, 'score': 'not_relevant'}}, 'dataset_stats': {}}
CPU times: user 102 ms, sys: 27.7 ms, total: 130 ms
Wall

# Diabetic Test

In [12]:
%%time
df = pd.read_csv("data/diabetic.csv")
df = df.fillna(0)
df.name = "diabetic"
del df["Notes"]
del df["ID"]
cat_cols = ["week","weekday"]
for c in cat_cols:
    df[c] = df[c].astype("category")
df.head()
repo = Reporter(df, profile="patient", domain_path="./domain_storage/", human_knowledge_path="./human_storage/")


      ['id', 'age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc', 'smoke', 'alco', 'active', 'cardio'] ['week', 'weekday', 'M0700', 'M0930', 'M1300', 'M1500', 'M1800', 'M2000', 'M2300', 'i0700', 'i0930', 'i1300', 'i1500', 'i1800', 'i2300']
      ['week', 'weekday', 'M0700', 'M0930', 'M1300', 'M1500', 'M1800', 'M2000', 'M2300', 'i0700', 'i0930', 'i1300', 'i1500', 'i1800', 'i2300'] ['week', 'weekday', 'M0700', 'M0930', 'M1300', 'M1500', 'M1800', 'M2000', 'M2300', 'i0700', 'i0930', 'i1300', 'i1500', 'i1800', 'i2300']
{'column_stats': {'week': {'value': 0.0, 'score': 'not_relevant'}, 'weekday': {'value': 0.0, 'score': 'not_relevant'}, 'M0700': {'value': 0.0, 'score': 'not_relevant'}, 'M0930': {'value': 0.0, 'score': 'not_relevant'}, 'M1300': {'value': 0.0, 'score': 'not_relevant'}, 'M1500': {'value': 0.0, 'score': 'not_relevant'}, 'M1800': {'value': 0.0, 'score': 'not_relevant'}, 'M2000': {'value': 0.0, 'score': 'not_relevant'}, 'M2300': {'value': 0.0, 'score': 'no

KeyError: 'color'

In [13]:
repo.report

{'columns': {'active': {'score': 'TODO', 'value': 0.0},
  'age': {'score': 'TODO', 'value': 0.0},
  'alco': {'score': 'TODO', 'value': 0.0},
  'ap_hi': {'score': 'TODO', 'value': 0.0},
  'ap_lo': {'score': 'TODO', 'value': 0.0},
  'cardio': {'score': 'TODO', 'value': 0.0},
  'cholesterol': {'score': 'TODO', 'value': 0.0},
  'gender': {'score': 'TODO', 'value': 0.0},
  'gluc': {'score': 'TODO', 'value': 0.0},
  'height': {'score': 'TODO', 'value': 0.0},
  'id': {'score': 'TODO', 'value': 0.0},
  'smoke': {'score': 'TODO', 'value': 0.0},
  'weight': {'score': 'TODO', 'value': 0.0}},
 'columns_profile': {'active': {'score': 'TODO', 'value': 0.3},
  'age': {'score': 'TODO', 'value': 6758.129000000001},
  'alco': {'score': 'TODO', 'value': 0.0},
  'ap_hi': {'score': 'TODO', 'value': 27.6},
  'ap_lo': {'score': 'TODO', 'value': 18.400000000000002},
  'cardio': {'score': 'TODO', 'value': 0.0},
  'cholesterol': {'score': 'TODO', 'value': 0.3},
  'gender': {'score': 'TODO', 'value': 0.245},
  '

# Heart test

In [14]:
%%time
df = pd.read_csv("data/heart.csv")

df.name = "heart"
cat_cols = ["sex","cp", "fbs", "restecg", "exang", "slope", "ca", "thal", "target"]
for c in cat_cols:
    df[c] = df[c].astype("category")
df.head()
repo = Reporter(df, profile="patient", domain_path="./domain_storage/", human_knowledge_path="./human_storage/")


      ['id', 'age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc', 'smoke', 'alco', 'active', 'cardio'] ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']
      ['week', 'weekday', 'M0700', 'M0930', 'M1300', 'M1500', 'M1800', 'M2000', 'M2300', 'i0700', 'i0930', 'i1300', 'i1500', 'i1800', 'i2300'] ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']
      ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'] ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']
{'column_stats': {'age': {'value': 0.0, 'score': 'not_relevant'}, 'sex': {'value': 0.0, 'score': 'not_relevant'}, 'cp': {'value': 0.0, 'score': 'not_relevant'}, 'trestbps': {'value': 0.0, 'score': 'not_relevant'}, 'chol': {'value':

KeyError: 'profile_knowledge'

In [15]:
repo.report

{'columns': {'active': {'score': 'TODO', 'value': 0.0},
  'age': {'score': 'TODO', 'value': 0.0},
  'alco': {'score': 'TODO', 'value': 0.0},
  'ap_hi': {'score': 'TODO', 'value': 0.0},
  'ap_lo': {'score': 'TODO', 'value': 0.0},
  'cardio': {'score': 'TODO', 'value': 0.0},
  'cholesterol': {'score': 'TODO', 'value': 0.0},
  'gender': {'score': 'TODO', 'value': 0.0},
  'gluc': {'score': 'TODO', 'value': 0.0},
  'height': {'score': 'TODO', 'value': 0.0},
  'id': {'score': 'TODO', 'value': 0.0},
  'smoke': {'score': 'TODO', 'value': 0.0},
  'weight': {'score': 'TODO', 'value': 0.0}},
 'columns_profile': {'active': {'score': 'TODO', 'value': 0.3},
  'age': {'score': 'TODO', 'value': 6758.129000000001},
  'alco': {'score': 'TODO', 'value': 0.0},
  'ap_hi': {'score': 'TODO', 'value': 27.6},
  'ap_lo': {'score': 'TODO', 'value': 18.400000000000002},
  'cardio': {'score': 'TODO', 'value': 0.0},
  'cholesterol': {'score': 'TODO', 'value': 0.3},
  'gender': {'score': 'TODO', 'value': 0.245},
  '

# Frontend module
Logic-free, just turns an information json to a user-readable report with images, which then prints out for the user to see.

In [16]:
class TextTemplates:
    def __init__(self):
    def red_json(self,):
PRETTY_JSON = {
    "graphs": {
        "some_col":{
            "type" : "bars",
            "cat_colors" : {
                "cat" : "red",
                "dog" : "blue",
            },
            "bg_color" : "white"
        }
    },
    "text" : {
        "some_col_avg" : {
            "reason" : "high",
            "number_ds" : 1.3,
            "number_domain": 0.2
        }
    }
}

IndentationError: expected an indented block (<ipython-input-16-0b67c57123be>, line 3)

In [ ]:
GRAPH_CLASSES = {
    "bars" : "asdasd",
    "pie" : "asdasd",
    "timeline" : ""
    #etc
}
class FrontEnd:
    def __init__(self, config):
        """Self explaining, associate graph with """
        #self.graphs_config = config["graphs"]
       # self.text_config = config["text"]
    def generate_graphs(self):
        for config in self.graphs_config:
            print(config)
    def generate_text(self):
        for config in self.text_config:
            print()
        #text things
        #graphical things
        pass

In [ ]:
class TournamentMaker():
    def __init__(self,seed, frontender): pass
    def present_candidate(self,report):pass #
    def organize_tournament(self):pass # generate tree
    def set_votes(self,votes):pass
    

In [ ]:
EXEC_PARAMS = {
    "domain_name" : "animals",
    "attribute_list": [] ,
    "knowledge" : {}
}
if __name__ == "__main__":
    """First we read our data"""
    df = pd.read_csv("nonsense.csv")
    #Do everything from the reporter class to get the report
    #Feed the report to the frontend class for the pretty representation
    print("henlo")